In [1]:
import He2021_v2_4 as H
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import matplotlib
import cv2
from astropy.io import fits
import pandas as pd
import random as r
import lenstronomy.Util.constants as c
import png
import pylab
import compose_class
from astropy.cosmology import Planck18 as cosmo
from lenstronomy.Cosmo.lens_cosmo import LensCosmo

import matplotlib.pyplot as plt
from lenstronomy.LensModel.lens_model import LensModel
import lenstronomy.Util.param_util as param_util
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

## construct a lensing system

In [2]:
#import lens and source catalgue（in this version, only 100 CMASS galaxies is used for the foreground lenses）
cmass_df = pd.read_csv('/nfsdata/users/zizhao-tulip/cata_Dec/decals_dev_cmass_dropnan_colrename_first100.csv')
eBOSS_df = pd.read_csv('/nfsdata/users/zizhao-tulip/cata_Dec/eBOSSDR12_QSO_catlogue_OnlyQSO.csv')

cmass_df = cmass_df[cmass_df["redshift"]>0]
eBOSS_df = eBOSS_df[eBOSS_df["Z_PIPE"]>0]

num_of_eBOSS = len(eBOSS_df)
num_of_cmass = len(cmass_df)

eBOSS_df.index = range(0,num_of_eBOSS)
cmass_df.index = range(0,num_of_cmass)

flag_abondon = 0

#set the boundary of lens/source plane
scale = 10
xmin = -scale
xmax = scale
ymin = -scale
ymax = scale

#the simualation pixel size
pixel_size = 0.1

#the threshold of flux ratio between two images
magratio_threshold = 10


#centre of the foreground lens
center_x = 0.0
center_y = 0.0


#we only keep the pair and quads (can be modified if needed)
possible_img_num = [2,4]  

def WrtLne(Line,f,flag):
    f.writelines(Line)
    if flag:
        f.write('\n')

/tmp/ipykernel_1098165/1863079025.py:3: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  eBOSS_df = pd.read_csv('/nfsdata/users/zizhao-tulip/cata_Dec/eBOSSDR12_QSO_catlogue_OnlyQSO.csv')


In [3]:
for ii in range(0,10):
    
    flag_abondon = 0
    
    #select lens and source
    cmass_index = r.randint(0,num_of_cmass-1)
    eBOSS_index = r.randint(0,num_of_eBOSS-1)

    while eBOSS_df["Z_PIPE"][eBOSS_index]<cmass_df["redshift"][cmass_index]:  
        eBOSS_index = r.randint(0,num_of_eBOSS-1)


    sigma_v = H.Appmag2Sigmav(cmass_df["redshift"][cmass_index],
                           eBOSS_df["Z_PIPE"][eBOSS_index],
                           cmass_df["rg_DEV"][cmass_index],
                           cmass_df["fluxr"][cmass_index])

    ######################################################################
    lens_cosmo = LensCosmo(z_lens=cmass_df["redshift"][cmass_index], z_source=eBOSS_df["Z_PIPE"][eBOSS_index],                            cosmo=cosmo)
    theta_E = lens_cosmo.sis_sigma_v2theta_E(sigma_v/1000)[0][0]
    ######################################################################

    #Judging whether to reselect the background according to the Einstein radius calculation
    num_of_while = 0
 
    while theta_E < 0.5:
        eBOSS_index = r.randint(0,num_of_eBOSS-1)
        lens_cosmo = LensCosmo(z_lens=cmass_df["redshift"][cmass_index], z_source=eBOSS_df["Z_PIPE"][eBOSS_index],                            cosmo=cosmo)
        theta_E = lens_cosmo.sis_sigma_v2theta_E(sigma_v/1000)[0][0]
        print("due to small theta_E, the QSO is reselected.")
        num_of_while += 1
        if num_of_while > 50:
            print("this zLens is too high(", round(cmass_df["redshift"][cmass_index],2), ") to find a suitable QSO.")
            flag_abondon = 1
            break
    
    #If the program can't find it, abandon the loop and start the next one
    if flag_abondon:
        continue

    #Calculating the axis ratio
    e1 = cmass_df['shapedev_e1'][cmass_index]
    e2 = cmass_df['shapedev_e2'][cmass_index]
    e = np.sqrt(e1**2 + e2**2)
    qlens = (1-e)/(1+e)

    #Given position angle
    phi_deg = np.random.rand()*180

    #Combining the results of the above steps, the complete lens parameters are given
    lpar = np.array( [
        theta_E,   #eninstein radius
        center_x,  #xcenter
        center_y,  #ycenter
        phi_deg,   #position angle, +Y轴为0度
        qlens,     #axis ratio
    ])

    #Split the lens\source plane by pixel size
    plane_xx,plane_yy = np.mgrid[xmin:xmax+pixel_size:pixel_size, ymin:ymax+pixel_size:pixel_size]
    plane_xx[plane_xx==0] = 1e-8
    plane_yy[plane_yy==0] = 1e-8

    #Get caustic and critical
    miu_inversed = H.GetInversedMiu(plane_xx,plane_yy,pixel_size, lpar)
    critical_inner, critical_outer, caustic_inner, caustic_outer = H.GetCritialCastic(plane_xx,plane_yy, miu_inversed,lpar)

    while True:
        
        beta_ra, beta_dec = H.GetSourcePosition(caustic_outer)
        beta_ra = beta_ra
        beta_dec = beta_dec
        beta_ra_dec = np.array([beta_ra, beta_dec])

        
        plane_xx, plane_yy = np.mgrid[xmin:xmax+pixel_size:pixel_size, ymin:ymax+pixel_size:pixel_size]
        img_plane_grid = np.c_[plane_xx.ravel(),plane_yy.ravel()]
        tri_img_plane_Delaunay = Delaunay(img_plane_grid)
        tri_img_plane = img_plane_grid[tri_img_plane_Delaunay.simplices]
        tri_source_plane = H.TriangleImg2Source(tri_img_plane, lpar)
        index_satisfied = H.GetIndexOfSatisfiedTriangle(tri_source_plane, beta_ra_dec)
        index_satisfied = H.DropCntrImg(tri_img_plane, index_satisfied, pixel_size)
        
        cond1 = len(index_satisfied) not in possible_img_num
        if cond1:
            print('num of imgs is ', len(index_satisfied),' resample the beta x/y.')
            continue
            
        tri_source_plane_satisfied = tri_source_plane[index_satisfied]
        tri_img_plane_satisfied = tri_img_plane[index_satisfied]
        img_ra_dac_list = []
        for idx in range(len(tri_img_plane_satisfied)):
            (imgx_x, imgx_y), iter_times = H.GetImgPositionOfOneImg(tri_img_plane, index_satisfied[idx], lpar, beta_ra_dec)
            img_ra_dac_list.append([imgx_x, imgx_y])
        
        
        cond2 = ['Catch a bug', 'Catch a bug'] in img_ra_dac_list
        if cond2:
            print('position program report a bug, resample the beta x/y.')
            continue
        miu_list = []

        for img_x, img_y in img_ra_dac_list:
            miu_list.append(abs(H.GetMagAnlytcl(img_x, img_y, lpar)))
            
        cond3 = max(miu_list)/min(miu_list) > magratio_threshold
        if cond3:
            print('reach miuratio threshold, resample the beta x/y.')
            continue
        if (not cond1) & (not cond2) & (not cond3):
            break

    miu_arr = np.array(miu_list)
    miu_sort_arr = np.sort(np.array(miu_list))
    img_ra_dac_sort_arr = np.array([])
    for idx in range(0,len(miu_arr)):
        this_crdnt = np.array(img_ra_dac_list[np.where(miu_sort_arr[idx]==miu_arr)[0][0]])
        img_ra_dac_sort_arr = np.append(img_ra_dac_sort_arr, this_crdnt)
        
    #Aggregate existing information, export & write to file
    lens_para_log = str(ii+1)+"\n"
    lens_para_log = lens_para_log+"q:              "+str(lpar[4])+"\n"+"zd:             "+str(cmass_df["redshift"][cmass_index])+"\n"+                    "zs:             "+str(eBOSS_df["Z_PIPE"][eBOSS_index])+"\n"
    lens_para_log = lens_para_log+"rng:            "+str(lpar[0])+"\n"+"phi:            "+str(lpar[3])+"\n"
    lens_para_log = lens_para_log+"sgm:            "+str(sigma_v[0][0]/1000)+"\n"
    img_log = "img_crdnt:      "+str(img_ra_dac_sort_arr)+"\n"
    miu_log = "miu:            "+str(miu_sort_arr)+"\n"
    qso_log = "QSO_crdnt:      "+str(beta_ra_dec)+"\n"
    msg_bfr_err = lens_para_log + miu_log + qso_log + img_log
    
    print(msg_bfr_err)
    f=open("5000tst_230614.txt","a")
    WrtLne([],f,True)
    WrtLne(msg_bfr_err,f,False)
    f.close()

    #lenstronomy
    lens_model_list = ['SIE']
    lensModel = LensModel(lens_model_list=lens_model_list, z_lens=cmass_df["redshift"][cmass_index], z_source=eBOSS_df["Z_PIPE"][eBOSS_index])
    e1_new, e2_new = param_util.phi_q2_ellipticity(phi=(phi_deg+90)/180.0*np.pi, q=qlens)
    kwargs_sie = {'theta_E':theta_E , 'e1': e1_new, 'e2': e2_new, 'center_x': 0.0, 'center_y': 0.0}
    kwargs_lens = [kwargs_sie]

    solver = LensEquationSolver(lensModel)
    theta_ra, theta_dec = solver.image_position_from_source(beta_ra, beta_dec, kwargs_lens)

    miu_lnstrnmy_arr = np.array([])
    for ra,dec in zip(theta_ra, theta_dec):
        miu_lnstrnmy_arr = np.append(miu_lnstrnmy_arr, abs(lensModel.magnification(ra,dec, kwargs_lens)))
    miu_lnstrnmy_sort_arr = np.sort(miu_lnstrnmy_arr)
    img_ra_dac_lnstrnmy_sort_arr = []
    for idx in range(0,len(miu_lnstrnmy_sort_arr)):
        this_ra  = np.array(theta_ra[np.where(miu_lnstrnmy_sort_arr[idx]==miu_lnstrnmy_arr)[0][0]])
        this_dec = np.array(theta_dec[np.where(miu_lnstrnmy_sort_arr[idx]==miu_lnstrnmy_arr)[0][0]])
        img_ra_dac_lnstrnmy_sort_arr = np.append(img_ra_dac_lnstrnmy_sort_arr, [this_ra,this_dec])

    if len(miu_lnstrnmy_arr) == len(miu_arr):
        #Perform error calculations
        mag_err = abs(miu_lnstrnmy_sort_arr - miu_sort_arr).sum()/miu_lnstrnmy_sort_arr.sum()
        pos_err = np.sqrt(((img_ra_dac_lnstrnmy_sort_arr-img_ra_dac_sort_arr)**2).sum())/pixel_size
        tst_rslt = "miu_lnsr:       "+str(miu_lnstrnmy_sort_arr)+               "\nmag_err:        "+str(mag_err)+"\npstn_err(pxl):  "+str(pos_err)
    
        if mag_err>1 or pos_err>0.01:
            tst_rslt=tst_rslt+";error BEYOND thrshld"
        else:
            tst_rslt=tst_rslt+";NORMAL"
    else:
        tst_rslt = "Differed in nimg. \n"+\
                   "miu_lnsr:       "+str(miu_lnstrnmy_sort_arr)+"\n"+\
                   "img_crdnt_lnsr: "+str(img_ra_dac_lnstrnmy_sort_arr)
                    
    msg_ftr_err = tst_rslt
    
    
    #Aggregate error information, output & write to file
    print(msg_ftr_err)
    f=open("5000tst_230614.txt","a")
    WrtLne(msg_ftr_err,f,True)
    f.close()


/home/zizhao/palace/He2021_v2_4.py:656: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function TriangleImg2Source failed at nopython mode lowering due to: list(list(float64)<iv=None>)<iv=None> cannot be represented as a NumPy dtype
  @jit(nopython=numba_flag)
/home/zizhao/palace/He2021_v2_4.py:656: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "TriangleImg2Source" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "He2021_v2_4.py", line 660:
def TriangleImg2Source(tri_img_plane, lpar_local):
    <source elided>

    for index in range(len(tri_img_plane)):
    ^

  @jit(nopython=numba_flag)
/nfsdata/users/zizhao/soft/anaconda-geranium/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "TriangleImg2Source" was compiled in object mode without forceobj=True, but has lifted loops.

File "He2021_

1
q:              0.8284836659377677
zd:             0.56092
zs:             1.481886
rng:            0.9103650747488692
phi:            159.86577889959088
sgm:            247.17219843495212
miu:            [0.49724914 3.29323672]
QSO_crdnt:      [-0.12797281 -0.53355743]
img_crdnt:      [ 0.08399645  0.32066218 -0.28872528 -1.39975942]

miu_lnsr:       [0.4972488  3.29323873]
mag_err:        6.20258124286978e-07
pstn_err(pxl):  4.312016100829436e-06;NORMAL
2
q:              0.6595701112785233
zd:             0.51913
zs:             2.331204
rng:            0.7326734641912929
phi:            89.14252195319538
sgm:            197.5450496056125
miu:            [0.90749317 2.56697157]
QSO_crdnt:      [0.24429313 0.21902607]
img_crdnt:      [-0.3404513  -0.16949539  0.58097916  0.89142176]

miu_lnsr:       [0.90749201 2.5669651 ]
mag_err:        2.193072928192727e-06
pstn_err(pxl):  4.73412402263756e-06;NORMAL


/home/zizhao/palace/He2021_v2_4.py:245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  nearby_tri_indexs = np.array([fisrt_tri_index, second_tri_index, thrid_tri_index, np.array([index_focus])])


3
q:              0.674100469913684
zd:             0.58204
zs:             2.2457392000000005
rng:            1.3536161856102367
phi:            116.33069094736713
sgm:            277.8658246960748
miu:            [1.27891322 2.01539897]
QSO_crdnt:      [-0.44352378  0.65169865]
img_crdnt:      [ 0.40035146 -0.48955358 -1.17177811  1.87585119]

miu_lnsr:       [1.27891454 2.01540167]
mag_err:        1.2184617976511996e-06
pstn_err(pxl):  3.7361152606240346e-06;NORMAL
due to small theta_E, the QSO is reselected.
4
q:              0.8575651088739036
zd:             0.5705899999999999
zs:             2.1751094
rng:            0.6724065490149984
phi:            169.49318886724234
sgm:            195.84913869179326
miu:            [0.75965966 2.40420753]
QSO_crdnt:      [ 0.32701222 -0.23647065]
img_crdnt:      [-0.21299362  0.17841466  0.91560822 -0.5851476 ]

miu_lnsr:       [0.75965672 2.4042071 ]
mag_err:        1.0678049226825402e-06
pstn_err(pxl):  6.138528632303114e-06;NORMAL
5
q:  

## Calculation of timedelay

In [6]:
#mine prgram
H.TwoImgTimeDelay(img_ra_dac_sort_arr, lpar,\
          zd=cmass_df["redshift"][cmass_index], zs=eBOSS_df["Z_PIPE"][eBOSS_index])

(117.48813860812128, 0.0)

In [7]:
#lenstronomy
fermat_lstrnmy_A = lensModel.fermat_potential(img_ra_dac_lnstrnmy_sort_arr[0], img_ra_dac_lnstrnmy_sort_arr[1], kwargs_lens)
fermat_lstrnmy_B = lensModel.fermat_potential(img_ra_dac_lnstrnmy_sort_arr[2], img_ra_dac_lnstrnmy_sort_arr[3], kwargs_lens)
print(lens_cosmo.time_delay_units(fermat_lstrnmy_A)-\
      lens_cosmo.time_delay_units(fermat_lstrnmy_B))

117.48809764806813
